#데이터얻기/ 준비

In [1]:
import FinanceDataReader as fdr
from datetime import datetime, timedelta
import numpy as np

daysbefore = datetime.now() - timedelta(weeks=5)
daysbefore = daysbefore.strftime("%Y-%m-%d")

#데이터 얻기
longdf = fdr.DataReader('233740', daysbefore)
shortdf = fdr.DataReader('251340', daysbefore)


In [2]:
longdf.tail()

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2019-04-15,13330,13350,13105,13190,8271066,-0.004153
2019-04-16,13190,13245,12980,13225,8354169,0.002654
2019-04-17,13230,13300,13135,13230,6306569,0.000378
2019-04-18,13190,13195,12515,12625,10839245,-0.045729
2019-04-19,12750,12940,12725,12925,5579609,0.023762


In [3]:
#range 값
longdf['range'] = longdf['High'] - longdf['Low']
shortdf['range'] = shortdf['High'] - shortdf['Low']

#노이즈 계산
longdf['noise'] = 1-abs(longdf['Open']-longdf['Close'])/(longdf['High']-longdf['Low'])
shortdf['noise'] = 1-abs(shortdf['Open']-shortdf['Close'])/(shortdf['High']-shortdf['Low'])

#13일 노이즈 평균
longdf['noise13'] = longdf['noise'].rolling(window=13).mean()
shortdf['noise13'] = shortdf['noise'].rolling(window=13).mean()

# 마켓타이밍 적용(3,5,10,20일 이동평균)
windows = [3, 5, 10, 20]
for window in windows:
    longdf['close_ma{}'.format(window)] = longdf['Close'].rolling(window).mean()
    shortdf['close_ma{}'.format(window)] = shortdf['Close'].rolling(window).mean()

# 변동성 2% 제한
longdf['std2'] = np.where(longdf['range']/longdf['Close']*100 > 2, 2*longdf['Close']/longdf['range']/100, 1)
shortdf['std2'] = np.where(shortdf['range']/shortdf['Close']*100 > 2, 2*shortdf['Close']/shortdf['range']/100, 1)


In [6]:
import pandas as pd

In [12]:
excel_writer = pd.ExcelWriter('./변동성돌파.xlsx', engine='xlsxwriter')
longdf.to_excel(excel_writer, 'long')
shortdf.to_excel(excel_writer, 'short')
excel_writer.save()

In [11]:
# 이동평균 모멘텀 스코어
longdf['mascore'] = (np.where(longdf['Close'] > longdf['close_ma3'], 1, 0) + np.where(longdf['Close'] > longdf['close_ma5'], 1, 0) + \
                    np.where(longdf['Close'] > longdf['close_ma10'], 1, 0) + np.where(longdf['Close'] > longdf['close_ma20'], 1, 0))/4
shortdf['mascore'] = (np.where(shortdf['Close'] > shortdf['close_ma3'], 1, 0) + np.where(shortdf['Close'] > shortdf['close_ma5'], 1, 0) + \
                    np.where(shortdf['Close'] > shortdf['close_ma10'], 1, 0) + np.where(shortdf['Close'] > shortdf['close_ma20'], 1, 0))/4

In [13]:
# 돌파 진입값
long_enterprice = longdf.iloc[-1,0]+longdf.iloc[-2,6]*longdf.iloc[-2,8]
short_enterprice = shortdf.iloc[-1,0]+shortdf.iloc[-2,6]*shortdf.iloc[-2,8]

# 손절값: range/4
long_losscut = format((long_enterprice-(longdf.iloc[-2,6]/4)).astype(int),',')
short_losscut = format((short_enterprice-(shortdf.iloc[-2,6]/4)).astype(int),',')

#계좌 총 잔고
valance = 1113110

long_buyratio = 0.2 + 0.6*longdf.iloc[-2,13]*longdf.iloc[-2,14]

#최종 투입 금액
long_amt = valance*long_buyratio
short_amt = valance - long_amt

#사야할 주식 수
long_buy = format((long_amt/long_enterprice).astype(int),',')
short_buy = format((short_amt/short_enterprice).astype(int),',')

long_amt = format(long_amt.astype(int),',')
short_amt = format(short_amt.astype(int),',')
long_enterprice = format(long_enterprice.astype(int),',')
short_enterprice = format(short_enterprice.astype(int),',')

In [ ]:
'''알고자하는 변수
  레버리지 진입 가격: long_enterprice  사야할 주식 수: long_buy
  인버스   진입 가격: short_enterprice 사야할 주식 수: short_buy
'''

#텔레그램에 전송
#pip install python-telegram-bot --upgrade

import telegram   #텔레그램 모듈을 가져옵니다.

my_token = '888106364:AAGAa5TCf_YZFPYZHMtM2r7a7pKrxV_qbPs'   #토큰을 변수에 저장합니다.

bot = telegram.Bot(token = my_token)   #bot을 선언합니다.

#updates = bot.getUpdates()  #업데이트 내역을 받아옵니다.

#for u in updates :   # 내역중 메세지를 출력합니다.
#  print(u.message)

#chat_id = bot.getUpdates()[-1].message.chat.id #가장 최근에 온 메세지의 chat id를 가져옵니다

bot.sendMessage(chat_id = 668333141,text="레버리지 진입 가격: "+ long_enterprice)
bot.sendMessage(chat_id = 668333141,text="레버리지 사야할 주식 수: "+ long_buy)
bot.sendMessage(chat_id = 668333141,text="인버스 진입 가격: "+ short_enterprice)
bot.sendMessage(chat_id = 668333141,text="인버스 사야할 주식 수: "+ short_buy)
bot.sendMessage(chat_id = 668333141,text="레버리지 손절 가격: "+ long_losscut)
bot.sendMessage(chat_id = 668333141,text="인버스 손절 가격: "+ short_losscut)